# Session 13: Assignment 1 - Raghunath

## Read the following data set: 
    https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data 

In [1]:
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', skipinitialspace=True)

In [2]:
# rename columns with meaningful names
headers = ['age', 'workclass','fnlwgt', 'education','educationnum', 'maritalstatus','occupation','relationship','race','sex',
          'capitalgain','capitalloss', 'hoursperweek','nativecountry', 'lessThanEqualTo50K']
df.columns= headers

In [3]:
from sqlalchemy import Column, Integer, Float, Date, String, VARCHAR, BigInteger
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

Base = declarative_base()
class AdultData(Base):
    __tablename__ = 'adult_data'
     
    index = Column(BigInteger, primary_key=True)
    age = Column(BigInteger)
    workclass = Column(String)
    fnlwgt = Column(BigInteger)
    education = Column(String)
    educationnum = Column(BigInteger)
    maritalstatus = Column(String)
    occupation = Column(String)
    relationship = Column(String)
    race = Column(String)
    sex = Column(String)
    capitalgain = Column(BigInteger)
    capitalloss = Column(BigInteger)
    hoursperweek = Column(BigInteger)
    nativecountry = Column(String)
    lessThanEqualTo50K = Column(String)

    def __repr__(self):
        return "<adultdata(index='%s', age='%s', workclass='%s', education ='%s', occupation='%s', race='%s', gender='%s')>" % (
                             self.index, self.age, self.workclass, self.education, self.occupation, self.race, self.sex)


### 1.  Create an sqlalchemy engine using a sample from the data set 

In [4]:
# sample data set from top 50 records
adult_df = df.head(50).copy()
adult_df.head()

engine = create_engine('sqlite:///AdultData.db')
Base.metadata.create_all(engine)
adult_df.to_sql(con=engine, index_label='index', name=AdultData.__tablename__, if_exists='replace')

In [5]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

data = session.query(AdultData)
data.all()

[<adultdata(index='0', age='50', workclass='Self-emp-not-inc', education ='Bachelors', occupation='Exec-managerial', race='White', gender='Male')>,
 <adultdata(index='1', age='38', workclass='Private', education ='HS-grad', occupation='Handlers-cleaners', race='White', gender='Male')>,
 <adultdata(index='2', age='53', workclass='Private', education ='11th', occupation='Handlers-cleaners', race='Black', gender='Male')>,
 <adultdata(index='3', age='28', workclass='Private', education ='Bachelors', occupation='Prof-specialty', race='Black', gender='Female')>,
 <adultdata(index='4', age='37', workclass='Private', education ='Masters', occupation='Exec-managerial', race='White', gender='Female')>,
 <adultdata(index='5', age='49', workclass='Private', education ='9th', occupation='Other-service', race='Black', gender='Female')>,
 <adultdata(index='6', age='52', workclass='Self-emp-not-inc', education ='HS-grad', occupation='Exec-managerial', race='White', gender='Male')>,
 <adultdata(index='

### 2.  Write two basic update queries 

In [6]:
from sqlalchemy import update
adult = session.query(AdultData).filter_by(index=8).first()

print("Record with index {0} before update: {1}".format(adult.index, adult))
adult.age = adult.age + 1
session.commit()
updated = session.query(AdultData).filter_by(index=8).first()
print("Record with index {0} after updating age: {1}".format(updated.index, updated))

Record with index 8 before update: <adultdata(index='8', age='42', workclass='Private', education ='Bachelors', occupation='Exec-managerial', race='White', gender='Male')>
Record with index 8 after updating age: <adultdata(index='8', age='43', workclass='Private', education ='Bachelors', occupation='Exec-managerial', race='White', gender='Male')>


In [7]:
session.query(AdultData).filter(AdultData.occupation == 'Adm-clerical').\
update({AdultData.occupation:"Admin-clerical"}, synchronize_session = False)

session.query(AdultData).filter(AdultData.occupation == 'Admin-clerical').all()

[<adultdata(index='11', age='23', workclass='Private', education ='Bachelors', occupation='Admin-clerical', race='White', gender='Female')>,
 <adultdata(index='32', age='30', workclass='Federal-gov', education ='Some-college', occupation='Admin-clerical', race='White', gender='Male')>,
 <adultdata(index='36', age='19', workclass='Private', education ='HS-grad', occupation='Admin-clerical', race='White', gender='Female')>,
 <adultdata(index='42', age='49', workclass='Private', education ='HS-grad', occupation='Admin-clerical', race='White', gender='Female')>]

### 3.  Write two delete queries

In [8]:
print("Before Delete :\n {0}".format(session.query(AdultData).filter_by(index=42).first()))
session.query(AdultData).filter_by(index=42).delete()

session.commit()
print("\nAfter Delete :\n {0}".format(session.query(AdultData).filter_by(index=42).first()))

Before Delete :
 <adultdata(index='42', age='49', workclass='Private', education ='HS-grad', occupation='Admin-clerical', race='White', gender='Female')>

After Delete :
 None


In [9]:
print("Before Delete :\n {0}".format(session.query(AdultData).filter(AdultData.age == 31).all()))
session.query(AdultData).filter(AdultData.age == 31).delete()

session.commit()
print("\nAfter Delete: \n {0}".format(session.query(AdultData).filter(AdultData.age == 31).all()))

Before Delete :
 [<adultdata(index='7', age='31', workclass='Private', education ='Masters', occupation='Prof-specialty', race='White', gender='Female')>, <adultdata(index='37', age='31', workclass='Private', education ='Some-college', occupation='Sales', race='White', gender='Male')>, <adultdata(index='39', age='31', workclass='Private', education ='9th', occupation='Machine-op-inspct', race='White', gender='Male')>]

After Delete: 
 []


### 4.  Write two filter queries 

In [10]:
from sqlalchemy import or_
#logical or
session.query(AdultData).filter(or_(AdultData.occupation == 'Farming-fishing', AdultData.occupation == 'Tech-support')).all()

[<adultdata(index='15', age='25', workclass='Self-emp-not-inc', education ='HS-grad', occupation='Farming-fishing', race='White', gender='Male')>,
 <adultdata(index='21', age='35', workclass='Federal-gov', education ='9th', occupation='Farming-fishing', race='Black', gender='Male')>,
 <adultdata(index='23', age='59', workclass='Private', education ='HS-grad', occupation='Tech-support', race='White', gender='Female')>,
 <adultdata(index='24', age='56', workclass='Local-gov', education ='Bachelors', occupation='Tech-support', race='White', gender='Male')>,
 <adultdata(index='41', age='24', workclass='Private', education ='Bachelors', occupation='Tech-support', race='White', gender='Male')>]

In [11]:
from sqlalchemy import and_
#logical and
session.query(AdultData).filter(and_(AdultData.occupation == 'Exec-managerial', AdultData.workclass != 'Private')).all()

[<adultdata(index='0', age='50', workclass='Self-emp-not-inc', education ='Bachelors', occupation='Exec-managerial', race='White', gender='Male')>,
 <adultdata(index='6', age='52', workclass='Self-emp-not-inc', education ='HS-grad', occupation='Exec-managerial', race='White', gender='Male')>,
 <adultdata(index='18', age='43', workclass='Self-emp-not-inc', education ='Masters', occupation='Exec-managerial', race='White', gender='Female')>]

### 5.  Write two function queries 

In [12]:
from sqlalchemy import func
session.query(func.count(AdultData.sex), AdultData.education).group_by(AdultData.education).all()

[(4, '11th'),
 (1, '7th-8th'),
 (2, '9th'),
 (3, 'Assoc-acdm'),
 (3, 'Assoc-voc'),
 (10, 'Bachelors'),
 (1, 'Doctorate'),
 (12, 'HS-grad'),
 (3, 'Masters'),
 (7, 'Some-college')]

In [13]:
session.query(func.count(AdultData.index)).scalar()

46

In [15]:
session.query(func.avg(AdultData.age)).scalar()

38.23913043478261